# In this lab, we will understand about various <font color="red">Runnables

In [ ]:
pip install langchain langchain_core langchain_openai

If you are using Jupyter notebook, follow the below instructions. Else skip this step and go to next step

**Open .env file in this folder and observe that we have configured OPENAI_API_KEY. Replace it with your own key or key given by me**

The Code in the below cell will load the .env file and set environment variables.

**Write the code in the below cell and execute it**

In [ ]:
from dotenv import load_dotenv

load_dotenv()


**Create llm object and set debug to true so that we can see debug logs**

In [ ]:

from langchain_openai import  ChatOpenAI
llm = ChatOpenAI()

from langchain_core.globals import set_debug
set_debug(False)

**If we want to Create a runnable object using a function or lambda, we can use RunnableLambda as Below.**


**Execute the below code in a cell to understand its usage**



In [ ]:
from langchain_core.runnables import RunnableLambda
r=RunnableLambda(lambda x: x + 1)
r.invoke(1)

**Below code is creating a Sequence of Runnable Lambdas**

**Execute the below code in a cell to understand its usage**

In [ ]:

from langchain_core.runnables import  RunnableParallel, RunnablePassthrough, RunnableSequence, RunnableLambda,RunnablePick,RunnableAssign

# A RunnableSequence constructed using the `|` operator
sequence = RunnableLambda(lambda x: x + 1) | RunnableLambda(lambda x: x * 2)
sequence.invoke(1) # 4
sequence.batch([1, 2, 3]) # [4, 6, 8]


**Below code shows how to execute  Runnables Parallelely using LCEL**

**Observe the debug logs carefully**

In [ ]:
from langchain_core.runnables import  RunnableParallel
rp= RunnableParallel({
    'mul_2': RunnableLambda(lambda x: x * 2),
    'mul_5': RunnableLambda(lambda x: x * 5)
})

rp.invoke(1)

In [ ]:

# A sequence that contains a RunnableParallel constructed using a dict literal
sequence = RunnableLambda(lambda x: x + 1) | {
    'mul_2': RunnableLambda(lambda x: x * 2),
    'mul_5': RunnableLambda(lambda x: x * 5)
}
sequence.invoke(1) # {'mul_2': 4, 'mul_5': 10}


**Let us understand about  RunnableParallel**

Use the below code and execute it in a cell




In [ ]:
from langchain.prompts import PromptTemplate
text = f"""
Needed a nice lamp for my bedroom, and this one had
additional storage and not too high of a price point.
Got it fast. The string to our lamp broke during the
transit and the company happily sent over a new one.
Came within a few days as well. It was easy to put
together. I had a missing part, so I contacted their
support and they very quickly got me the missing piece!
Lumina seems to me to be a great company that cares
about their customers and products!!"""


summarizarion_prompt = "summarize the below text into one sentence: {text}"

sentiment_prompt = "What is the sentiment of the  text below :  {text}"

fprompt = PromptTemplate(  template=summarizarion_prompt, input_variables=["text"])

sprompt = PromptTemplate(  template=sentiment_prompt,input_variables=["text"])

summarychain = fprompt | llm
sentimentchain = sprompt | llm

finalchain = RunnableParallel({"summary": summarychain, "sentiment": sentimentchain})
finalchain.invoke({"text": text})

**Let us understand about  RunnablePassthrough**

RunnablePassthrough is a type of runnable which will just give input as output

Execute the below code in a cell and observe the result


In [ ]:
rp= RunnablePassthrough()
rp.invoke({"language": "java", "task": "return a sum of numbers in a list. Do proper formatting of result"})

**Below is another example usage of RunnableParallel and RunnablePassthrough.**

**Execute the below code in a cell and observe the result**


In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

**Let us understand about  RunnablePick**


In [ ]:
r=RunnablePick("language")

r.invoke({"language": "java", "task": "return a sum of numbers in a list. Do proper formatting of result"})

**Let us understand about  RunnablePassthrough.assign or RunnableAssign**

Understand and execute the code in the  below cell


In [ ]:
from langchain_core.runnables.passthrough import RunnableAssign, RunnableParallel
def add_ten(x):
    return  x["input"] + 10

mapper ={"add_ten": add_ten}
runnable_assign = RunnableAssign(mapper)
print(runnable_assign)

output = runnable_assign.invoke({"input": 5})
output

**Understand and execute the code in the  below cell**


In [ ]:
from langchain_core.runnables.passthrough import RunnableAssign, RunnablePassthrough

ra=RunnableAssign(mapper={"d": RunnablePassthrough()})
ra.invoke({"language": "java", "task": "return a sum of numbers in a list. Do proper formatting of result"})

**Understand and execute the code in the  below cell**

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

def multiply_by_two(x):
    return  x["num"] * 2


r= RunnablePassthrough.assign(mult= multiply_by_two )
print( r)

r.invoke({"num": 1})


**Understand and execute the code in the  below cell**

In [ ]:
runnable = RunnableParallel(
    extra=RunnablePassthrough.assign(mult= multiply_by_two),
    modified=lambda x: x["num"] + 1,
)
print(runnable)
runnable.invoke({"num": 1})